In [3]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime 
from matplotlib.colors import rgb2hex

try:
    conn = psycopg2.connect("dbname='fantasysheets' user='postgres' host='viaduct.proxy.rlwy.net' password='WIKrjPIYtqCWApMIXculsqbMIQcGotEg' port='38391'")
except:
    print("I am unable to connect to the database")

# Execute the query and fetch data
cur = conn.cursor() 

query = """
with floor_ceiling as (
SELECT 
  hero_id,
  PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY seven_day_fantasy_score) AS floor_score,
  PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY seven_day_fantasy_score) AS ceiling_score
FROM (
  SELECT hero_id,start_datetime::date as start_date, avg(seven_day_fantasy_score) as seven_day_fantasy_score
  FROM flatten.GET_HEROS_WITH_STATS_SNAPSHOT
  WHERE start_datetime >= NOW() AT TIME ZONE 'UTC' - INTERVAL '28 days'
  and is_deleted = 0
  GROUP BY 1,2
) combined_data
GROUP BY hero_id
)
, RankedTrades as (
    SELECT
        lt.hero_rarity_id,
        lt.hero_handle,
        lt.hero_id,
        lt.price,
        lt.rarity,
        lt.timestamp,
        lt.db_updated_cst,
        ROW_NUMBER() OVER (PARTITION BY lt.hero_rarity_id ORDER BY lt.timestamp DESC) AS rn,
        COUNT(*) OVER (PARTITION BY lt.hero_rarity_id) AS total_count
    FROM  flatten.get_hero_last_trades lt
    WHERE  1=1
        and lt.rarity  in ('common','rare')
        and lt.price > .0035 --lowest  floor, avoid bad sales data
       AND timestamp >= NOW() AT TIME ZONE 'UTC' - INTERVAL '14 days'
)
SELECT 
        trades.rarity,
        trades.hero_handle,
        floor_score,
        ceiling_score,
        ROUND(AVG(CASE WHEN rn IN (1,2,3) THEN price ELSE NULL END),5) AS last_three_avg,
        ROUND(AVG(CASE WHEN rn IN (3,4,5) THEN price ELSE NULL END),5) AS prior_three_avg,    
        ROUND(MIN(case when rn in (1)  then PRICE else null end),5) as last_price,
        MIN(price) as min_price,
        ROUND(PERCENTILE_CONT(.03) WITHIN GROUP(ORDER BY price)::NUMERIC,5) as "30pct_price",
        MAX(price) AS max_price,
        MAX(trades.timestamp) AS last_trade,
        MAX(trades.db_updated_cst) AS db_updated_cst
    FROM RankedTrades trades
    left join floor_ceiling
    on trades.hero_id = floor_ceiling.hero_id
    where rarity in ('rare','common')
    and timestamp >= NOW() AT TIME ZONE 'UTC' - INTERVAL '14 days'
    GROUP BY 
        1,2,3,4
"""

cur.execute(query)
data = cur.fetchall()

# Create DataFrame with column names
columns = ['rarity', 'hero_handle', 'floor_score', 'ceiling_score', 
           'last_three_avg', 'prior_three_avg', 'last_price', 
           'min_price', 'thirty_percentile', 'max_price', 
           'last_trade', 'db_updated_cst']

df = pd.DataFrame(data, columns=columns)

# Display the DataFrame
print(df)

     rarity   hero_handle  floor_score  ceiling_score last_three_avg  \
0    common    0xBreadguy   501.222433     859.801345        0.01554   
1    common      0xDamien   379.831758     580.871996        0.00609   
2    common        0xENAS   831.353098     889.891732        0.07556   
3    common      0xfoobar   160.715918     571.924199        0.00563   
4    common        0xgaut   251.269992     477.678877        0.01395   
..      ...           ...          ...            ...            ...   
347    rare        zacxbt   631.520552     777.953123        0.10670   
348    rare      Zagabond   573.288599     742.863916        0.04300   
349    rare        Zeneca   435.341683     607.929684        0.03346   
350    rare         zhusu   325.393049     619.478989        0.03484   
351    rare  ZoomerOracle   278.137416     457.422791        0.01907   

    prior_three_avg last_price               min_price thirty_percentile  \
0           0.01975    0.01210  0.00615000000000000000     

In [5]:
# ... existing imports and database query ...

# Split and rename columns like before
rare_df = df[df['rarity'] == 'rare'][['hero_handle', 'ceiling_score', 'last_three_avg', 
                                     'prior_three_avg', 'last_price', 'min_price', 'thirty_percentile']]
common_df = df[df['rarity'] == 'common'][['hero_handle', 'ceiling_score', 'last_three_avg', 
                                         'prior_three_avg', 'last_price', 'min_price', 'thirty_percentile']]

columns_rename = {
    'hero_handle': 'Hero',
    'ceiling_score': 'Ceiling',
    'last_three_avg': 'Last 3',
    'prior_three_avg': 'Prior 3',
    'last_price': 'Last',
    'min_price': 'Min',
    'thirty_percentile': '30%'
}

rare_df = rare_df.rename(columns=columns_rename)
common_df = common_df.rename(columns=columns_rename)

# Style function for numeric columns with NULL handling
def style_df(df):
    return df.style\
        .format({
            'Ceiling': lambda x: '{:.2f}'.format(x) if pd.notnull(x) else '-',
            'Last 3': lambda x: '{:.4f}'.format(x) if pd.notnull(x) else '-',
            'Prior 3': lambda x: '{:.4f}'.format(x) if pd.notnull(x) else '-',
            'Last': lambda x: '{:.4f}'.format(x) if pd.notnull(x) else '-',
            'Min': lambda x: '{:.4f}'.format(x) if pd.notnull(x) else '-',
            '30%': lambda x: '{:.4f}'.format(x) if pd.notnull(x) else '-'
        })\
        .set_properties(**{
            'text-align': 'center',
            'padding': '5px',
            'border': '1px solid gray'
        })\
        .set_table_styles([
            {'selector': 'th',
             'props': [('background-color', '#f2f2f2'),
                      ('text-align', 'center'),
                      ('padding', '5px'),
                      ('font-weight', 'bold')]},
            {'selector': 'caption',
             'props': [('text-align', 'center'),
                      ('font-size', '16px'),
                      ('font-weight', 'bold'),
                      ('padding', '8px')]}
        ])\
        .set_caption('Heroes Price Analysis')

# Display the styled tables
print("Rare Heroes:")
display(style_df(rare_df))
print("\nCommon Heroes:")
display(style_df(common_df))

Rare Heroes:


TypeError: unsupported format string passed to NoneType.__format__


Common Heroes:


TypeError: unsupported format string passed to NoneType.__format__